仮説　悪天候時には特定のアプリカテゴリーの使用が増加する。
悪天候の定義は,悪天候（高い降水量や低い気温）の日
この仮説に対して初め悪天候時に特定のアプリカテゴリの使用が増加するかどうかの仮説を検証します。

In [9]:

import sqlite3
import pandas as pd

conn_user_activity = sqlite3.connect('/Users/beetlea/Documents/DSP2F/dsp2-final/usage_data.db')

query_user_activity = "SELECT * FROM user_activity"

user_activity_data = pd.read_sql_query(query_user_activity, conn_user_activity)


conn_user_activity.close()

user_activity_data.head()

,id,user_name,date,entertainment,sns,utility,creativity,travel,work_efficiency_finance,games
0,1,A,2023-12-01,125,165,60,30,5,1,15
1,2,A,2023-12-02,123,143,30,20,4,1,13
2,3,A,2023-12-03,100,50,30,23,5,4,2
3,4,A,2023-12-04,109,54,23,35,5,4,21
4,5,A,2023-12-05,253,254,23,35,5,4,56


In [11]:

conn_weather = sqlite3.connect('/Users/beetlea/Documents/DSP2F/dsp2-final/weather_data.db')

query_weather = "SELECT * FROM weather_data"

weather_data = pd.read_sql_query(query_weather, conn_weather)

conn_weather.close()

weather_data.head()


,Day,Local Pressure,Sea Level Pressure,Total Precipitation,1h Precipitation,10min Precipitation,Average Temperature,Max Temperature,Min Temperature,Average Humidity,...,Average Wind Speed,Max Wind Speed,Max Wind Direction,Max Instant Wind Speed,Max Instant Wind Direction,Sunshine Duration,Snowfall,Deepest Snow,Daytime Weather,Night Weather
0,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,1,1012.3,1015.2,--,--,--,8.7,12.6,6.4,54,...,2.0,4.0,南南東,7.4,北,5.4,--,--,晴時々曇,快晴
3,2,1015.6,1018.5,--,--,--,8.3,13.1,4.1,60,...,1.7,3.1,東,5.1,西北西,8.7,--,--,晴後一時曇,晴
4,3,1015.5,1018.5,--,--,--,9.1,15.2,4.1,58,...,2.5,6.3,北西,10.5,北西,8.3,--,--,晴,快晴


In [13]:

weather_data['date'] = pd.to_datetime(weather_data['Day'] + '-12-2023', errors='coerce', format='%d-%m-%Y')

weather_data['Total Precipitation'] = pd.to_numeric(weather_data['Total Precipitation'], errors='coerce')

weather_data = weather_data[['date', 'Total Precipitation', 'Average Temperature']]

merged_data = pd.merge(user_activity_data, weather_data, on='date', how='left')

merged_data.head()


,id,user_name,date,entertainment,sns,utility,creativity,travel,work_efficiency_finance,games,Total Precipitation,Average Temperature
0,1,A,2023-12-01,125,165,60,30,5,1,15,NaN,8.7
1,2,A,2023-12-02,123,143,30,20,4,1,13,NaN,8.3
2,3,A,2023-12-03,100,50,30,23,5,4,2,NaN,9.1
3,4,A,2023-12-04,109,54,23,35,5,4,21,NaN,8.7
4,5,A,2023-12-05,253,254,23,35,5,4,56,0.0,8.1


In [14]:

user_activity_data['date'] = pd.to_datetime(user_activity_data['date'], format='%Y-%m-%d')

merged_data = pd.merge(user_activity_data, weather_data, on='date', how='left')

merged_data.head()


,id,user_name,date,entertainment,sns,utility,creativity,travel,work_efficiency_finance,games,Total Precipitation,Average Temperature
0,1,A,2023-12-01,125,165,60,30,5,1,15,NaN,8.7
1,2,A,2023-12-02,123,143,30,20,4,1,13,NaN,8.3
2,3,A,2023-12-03,100,50,30,23,5,4,2,NaN,9.1
3,4,A,2023-12-04,109,54,23,35,5,4,21,NaN,8.7
4,5,A,2023-12-05,253,254,23,35,5,4,56,0.0,8.1


分析では、天気データ（特に「Total Precipitation」（降水量）と「Average Temperature」（平均気温））とアプリカテゴリの使用量の
間の関連を調べます。特に、降水量が高い日や気温が低い日にアプリの使用が増えるかどうかを分析します。

分析の結果、悪天候の日と晴れの日における各アプリカテゴリの使用量の平均を比較したデータが得られました。以下がその比較です：

エンターテイメント: 悪天候時の平均使用量は 194.8、晴れ時は 195.16
SNS: 悪天候時の平均使用量は 156.4、晴れ時は 152.35
ユーティリティ: 悪天候時の平均使用量は 37.8、晴れ時は 39.49
クリエイティビティ: 悪天候時の平均使用量は 27.8、晴れ時は 25.97
トラベル: 悪天候時の平均使用量は 6.5、晴れ時は 6.88
ワーク効率/ファイナンス: 悪天候時の平均使用量は 5.4、晴れ時は 5.06
ゲーム: 悪天候時の平均使用量は 42.6、晴れ時は 43.8
これらの結果から、悪天候と晴れの日との間でアプリカテゴリの使用量には小さな違いがあることがわかります。ただし、これらの違いが統計的に有意かどうかを判断するには、さらなる統計的検定が必要です。現段階では、天候とアプリの使用量との間には顕著な相関関係は見られまでした。
